In [3]:
import pandas as pd
import numpy as np
import datetime
import gc


In [4]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above

In [5]:
def fix_str_float(ds, col):
    ds[col] = ds[col].str.replace(r'[^0-9\.]','')
    ds[col] = np.where(ds[col]=='',np.nan,ds[col])
    ds[col] = ds[col].astype('float32')
    return ds[col].astype('float32')

In [6]:
def remove_unused_cols(X):
    #remove empty columns
    to_drop = []
    for col in X.select_dtypes(include='number').columns:
        if X[col].sum()==0:
            to_drop.append(col)
    if len(to_drop)>0:
        X = X.drop(columns=to_drop)
    return X

In [7]:
def preprocess(self, X , y = None ):
    # X = X.copy() # do we need this line
    # boolean transformations
    if 'event_uuid' in X.columns:
        X['event_uuid'] = np.where(X['event_uuid'].isnull(), 0,1)
    if 'click_hash' in X.columns:
        X['click_hash'] = np.where(X['click_hash'].isnull(), 0,1)
    if 'Android' in X.columns:
        X['Android'] = np.where(X['user_agent'].str.contains('Android', regex=False),1,0)
    if 'iOS' in X.columns:
        X['iOS'] = np.where(X['user_agent'].str.contains('Darwin', regex=False) | X['user_agent'].str.contains('iOS', regex=False),1,0)
    if 'trans_id' in X.columns:
        X['trans_id'] = np.where(X['trans_id'].isnull(), 0,1)
    # date transformations
    if 'created' in X.columns:
        X['created_weekday'] = X['created'].dt.weekday
        X['created_hour'] = X['created'].dt.hour
        X['created_minute'] = X['created'].dt.minute
    if 'date' in X.columns:
        X['date_weekday'] = X['date'].dt.weekday
        X['date_hour'] = X['date'].dt.hour
        X['date_minute'] = X['date'].dt.minute
        X['date_second'] = X['date'].dt.second
    #remove unused columns
    to_drop = []
    for col in ['date','created', 'install_diff','device_brand','install_seconds','user_agent','device_id']:
        if col in X.columns:
            to_drop.append(col)
    X = X.drop(columns=to_drop)
    X = pd.get_dummies(X, dummy_na=True, prefix_sep='=')
    #returns numpy array
    return X

    

In [40]:
#read from sample
convertions_ids_sample = pd.read_csv('data/convertions_ids_sample.csv', dtype={'ref_hash':'object'})['ref_hash']

In [41]:
# auctions
#auctions_df = pd.read_csv('data/auctions.csv', low_memory=False, dtype={'country':'category','platform':'category',\
#                                                                        'ref_type_id':'category','source_id':'category','device_id':'object'})
#auctions_df['date'] = pd.to_datetime(auctions_df['date'])
#auctions_sample = auctions_df.loc[auctions_df['device_id'].isin(auctions_ids_sample)].copy()
# installs
installs_df = pd.read_csv('data/installs.csv', low_memory=False, dtype={'ref_type':'category','application_id':'category',\
                                                      'device_brand':'category','ref_hash':'object','wifi':'category'})
installs_df['kind'] = installs_df['kind'].str.lower()
installs_df['kind'] = installs_df['kind'].astype('category')
installs_df.drop(columns=['session_user_agent','ip_address','device_language','device_model'], inplace=True)
installs_df['created'] = pd.to_datetime(installs_df['created'])
installs_df.drop(['device_countrycode'], axis=1, inplace=True)
installs_sample = installs_df.loc[installs_df['ref_hash'].isin(convertions_ids_sample)].copy()
#events
events_df = pd.read_csv('data/events.csv', low_memory=False, dtype={'event_id':'int32','ref_type':'category','application_id':'category',\
                                                                                            'attributed':'bool','device_countrycode':'category','device_city':'category',\
                                                                                            'trans_id':'category','carrier':'category','device_os':'category',\
                                                                                            'connection_type':'category'})
events_df['date'] = pd.to_datetime(events_df['date'])
events_df['wifi'].astype('bool', inplace=True)
events_df.drop(columns=['device_countrycode','session_user_agent','ip_address','device_language'], inplace=True)
events_sample = events_df.loc[events_df['ref_hash'].isin(convertions_ids_sample)].copy()
del events_df
#del auctions_df
del installs_df
gc.collect()
allDone()
print('setup done')

setup done


In [42]:
installs_sample.shape

(1020, 13)

In [43]:
events_sample.shape

(17319, 19)

In [44]:
installs_sample.dtypes

created           datetime64[ns]
application_id          category
ref_type                category
ref_hash                  object
click_hash                object
attributed                  bool
implicit                    bool
device_brand            category
user_agent                object
event_uuid                object
kind                    category
wifi                    category
trans_id                  object
dtype: object

In [110]:
installs_sample.dtypes

created            datetime64[ns]
application_id           category
ref_type                 category
ref_hash                   object
click_hash                 object
attributed                   bool
implicit                     bool
device_brand             category
user_agent                 object
event_uuid                 object
kind                     category
wifi                     category
trans_id                   object
in_seconds                float64
status_censored              bool
last_seen                 float64
dtype: object

In [117]:
# calculate time in seconds
installs_sample.drop_duplicates(inplace=True)
installs_sample = installs_sample.sort_values(by=['ref_hash','created'])
installs_sample['date_dif'] = installs_sample['created'].shift(periods=-1) - installs_sample['created']
installs_sample['device_id_next'] = installs_sample['ref_hash'].astype('object').shift(periods=-1)
installs_sample['date_dif'] = pd.to_timedelta(np.where(installs_sample['device_id_next']==installs_sample['ref_hash'], installs_sample['date_dif'], np.where(installs_sample['created']+pd.DateOffset(3)>'2019-04-27 00:00:00', datetime.datetime(2019,4,27)-installs_sample['created'], pd.to_timedelta(3, unit='d'))))
installs_sample['in_seconds'] = installs_sample['date_dif'].dt.total_seconds()
installs_sample['status_censored'] = ((installs_sample['device_id_next']==installs_sample['ref_hash']) | (installs_sample['in_seconds']==259200.0))
installs_sample.drop(['device_id_next','date_dif'], axis='columns', inplace=True)
installs_sample['ref_hash'] = installs_sample['ref_hash'].astype('object')
#calculate previous time in seconds
installs_sample['date_prev'] = installs_sample['created'].shift()
installs_sample['date_dif_prev'] = installs_sample['created']- installs_sample['date_prev']
installs_sample['device_id_prev'] = installs_sample['ref_hash'].astype('object').shift()
installs_sample['date_dif_prev'] = pd.to_timedelta(np.where(installs_sample['device_id_prev']==installs_sample['ref_hash'], installs_sample['date_dif_prev'], np.where(installs_sample['created']-pd.DateOffset(3)<'2019-04-18 00:00:00', installs_sample['created']-datetime.datetime(2019,4,18), pd.to_timedelta(3, unit='d'))))
installs_sample['last_seen'] = installs_sample['date_dif_prev'].dt.total_seconds()
installs_sample.drop(['device_id_prev','date_dif_prev','date_prev'], axis='columns', inplace=True)
installs_sample = installs_sample.sort_values(by=['created'])

#average time
installs_sample['hora'] = installs_sample['created'].dt.hour
installs_sample['dia'] = installs_sample['created'].dt.day
pivot_auctions = installs_sample.groupby(['dia','hora']).size().reset_index()
pivot_auctions.columns = ['dia','hora', 'hour_day_dist']
max_auctions = pivot_auctions['hour_day_dist'].max().max()
pivot_auctions['hour_day_dist'] = pivot_auctions['hour_day_dist']/max_auctions
display(pivot_auctions.head(5))
installs_sample = pd.merge(installs_sample, pivot_auctions, on=['dia','hora'], how='left')
installs_sample.drop(columns=['hora','dia'], inplace=True)




,dia,hora,hour_day_dist
0,18,0,0.3125
1,18,1,0.3125
2,18,2,0.7500
3,18,3,0.4375
4,18,4,0.4375


In [118]:
# calculate time in seconds
events_sample.drop_duplicates(inplace=True)
events_sample = events_sample.rename(columns = {'date':'created'}).sort_values(by=['ref_hash','created'])
events_sample['date_dif'] = events_sample['created'].shift(periods=-1) - events_sample['created']
events_sample['device_id_next'] = events_sample['ref_hash'].astype('object').shift(periods=-1)
events_sample['date_dif'] = pd.to_timedelta(np.where(events_sample['device_id_next']==events_sample['ref_hash'], events_sample['date_dif'], np.where(events_sample['created']+pd.DateOffset(3)>'2019-04-27 00:00:00', datetime.datetime(2019,4,27)-events_sample['created'], pd.to_timedelta(3, unit='d'))))
events_sample['in_seconds'] = events_sample['date_dif'].dt.total_seconds()
events_sample['status_censored'] = ((events_sample['device_id_next']==events_sample['ref_hash']) | (events_sample['in_seconds']==259200.0))
events_sample.drop(['device_id_next','date_dif'], axis='columns', inplace=True)
events_sample['ref_hash'] = events_sample['ref_hash'].astype('object')
#calculate previous time in seconds
events_sample['date_prev'] = events_sample['created'].shift()
events_sample['date_dif_prev'] = events_sample['created']- events_sample['date_prev']
events_sample['device_id_prev'] = events_sample['ref_hash'].astype('object').shift()
events_sample['date_dif_prev'] = pd.to_timedelta(np.where(events_sample['device_id_prev']==events_sample['ref_hash'], events_sample['date_dif_prev'], np.where(events_sample['created']-pd.DateOffset(3)<'2019-04-18 00:00:00', events_sample['created']-datetime.datetime(2019,4,18), pd.to_timedelta(3, unit='d'))))
events_sample['last_seen'] = events_sample['date_dif_prev'].dt.total_seconds()
events_sample.drop(['device_id_prev','date_dif_prev','date_prev'], axis='columns', inplace=True)
events_sample = events_sample.sort_values(by=['created'])
events_sample['user_agent']=np.nan
events_sample['kind']='event' #it is hashed

#average time
events_sample['hora'] = events_sample['created'].dt.hour
events_sample['dia'] = events_sample['created'].dt.day
pivot_auctions = events_sample.groupby(['dia','hora']).size().reset_index()
pivot_auctions.columns = ['dia','hora', 'hour_day_dist']
max_auctions = pivot_auctions['hour_day_dist'].max().max()
pivot_auctions['hour_day_dist'] = pivot_auctions['hour_day_dist']/max_auctions
display(pivot_auctions.head(5))
events_sample = pd.merge(events_sample, pivot_auctions, on=['dia','hora'], how='left')
events_sample.drop(columns=['hora','dia'], inplace=True)



,dia,hora,hour_day_dist
0,18,0,0.223502
1,18,1,0.066820
2,18,2,0.239631
3,18,3,0.195853
4,18,4,0.124424


In [111]:
events_sample.head()

,index,created,event_id,ref_type,ref_hash,application_id,attributed,device_os_version,device_brand,device_model,...,user_agent,event_uuid,carrier,kind,device_os,wifi,connection_type,in_seconds,status_censored,last_seen
3719932,239762,2019-04-18 00:01:06.622,15,1891515180541284343,1994349155897976914,65,False,NaN,6.115026e+18,7.620636e+18,...,NaN,3c4d9199-b38e-477e-a841-3baad5c62dc3,NaN,event,NaN,False,NaN,78.571,True,66.622
6109122,107026,2019-04-18 00:02:25.193,13,1891515180541284343,1994349155897976914,65,False,NaN,6.115026e+18,7.620636e+18,...,NaN,e29a0a43-082f-4a8e-acc4-b08343fdf9a5,NaN,event,NaN,False,NaN,6504.175,True,78.571
5444301,61073,2019-04-18 00:09:27.491,23,1891515180541284343,5113298256813305892,121,False,NaN,2.208835e+18,5.268998e+18,...,NaN,abd97705-5f77-441b-b068-14513951bc6a,NaN,event,NaN,False,NaN,48.578,True,567.491
474331,82303,2019-04-18 00:10:16.069,23,1891515180541284343,5113298256813305892,121,False,NaN,2.208835e+18,5.268998e+18,...,NaN,23111abd-b086-482e-8d83-178fdc829d96,NaN,event,NaN,False,NaN,26.040,True,48.578
476541,93398,2019-04-18 00:10:42.109,23,1891515180541284343,5113298256813305892,121,False,NaN,2.208835e+18,5.268998e+18,...,NaN,8e632562-7768-45d4-9362-1fddd2fe9df2,NaN,event,NaN,False,NaN,289.508,True,26.040


In [120]:
#some features
install_cols = installs_sample.columns.tolist()
print(1)
#information about last installs and events
convertions = installs_sample[['ref_hash','created','application_id','user_agent','wifi','kind','in_seconds','status_censored','last_seen','hour_day_dist']].append(events_sample[['ref_hash', 'created', 'application_id','user_agent','wifi','kind','in_seconds','status_censored','last_seen','hour_day_dist']], ignore_index=True)
print(convertions.shape)
print(2)

#user agent
convertions['user_agent'] = np.where(convertions['user_agent'].str.contains('Android', regex=False),'Android',convertions['user_agent'])
convertions['user_agent'] = np.where(convertions['user_agent'].str.contains('Darwin', regex=False) | convertions['user_agent'].str.contains('iOS', regex=False),'iOS',convertions['user_agent'])
print(3)

# previus applications

app_id_1 = convertions[['application_id','created','ref_hash']].copy()
group_1 = pd.merge(convertions, app_id_1, on='ref_hash', how='inner')
group_1 = group_1.loc[(group_1['created_x']>group_1['created_y']) | group_1['created_y'].isnull()]
group_1['application_id'] = group_1['application_id_y']
group_1['created'] = group_1['created_x']
group_1.drop(columns=['application_id_y','created_x', 'application_id_x'], inplace=True)
group_1 = group_1.loc[group_1['created_y']>group_1['created']-pd.DateOffset(7)]
group_1 = pd.get_dummies(group_1,prefix_sep='=', dummy_na=True,columns=['application_id'],drop_first=True)
app_id_1_columns = []
for col in group_1.columns.tolist():
    if col.startswith('application_id'):
        app_id_1_columns.append(col)
group_1 = group_1.groupby(['created','ref_hash']).agg({col:'sum' for col in app_id_1_columns})
convertions = pd.merge(convertions, group_1, on=['created','ref_hash'], how='left')
convertions['curr_app_id'] = convertions['application_id']
convertions.drop(columns=['application_id'], inplace=True)
print(4)



convertions = pd.get_dummies(convertions,prefix_sep='=', dummy_na=True,columns=['curr_app_id','wifi', 'kind', 'user_agent'],drop_first=True)

convertions.fillna(value={'application_id=nan':1,'kind=nan':1,'curr_app_id=nan':1}, inplace=True)
print(6)
convertions.fillna(value={col:0 for col in app_id_1_columns}, inplace=True)
print(7)
convertions = convertions.astype({col:'int32' for col in app_id_1_columns})
print(8)
convertions.reset_index(inplace=True, drop=True)
print(9)


1
(18339, 10)
2
3
4
6
7
8
9


In [121]:
convertions.head()

,ref_hash,created,in_seconds,status_censored,last_seen,hour_day_dist,application_id=116,application_id=117,application_id=121,application_id=122,...,curr_app_id=77,curr_app_id=78,curr_app_id=80,curr_app_id=81,curr_app_id=86,curr_app_id=87,curr_app_id=94,curr_app_id=95,curr_app_id=96,curr_app_id=nan
0,5113298256813305892,2019-04-18 00:09:26.652,259200.0,False,566.652,0.3125,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,138757233053411140,2019-04-18 00:11:33.745,259200.0,False,693.745,0.3125,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1770048835252550549,2019-04-18 00:25:37.349,259200.0,False,1537.349,0.3125,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4044669276886152554,2019-04-18 00:39:40.502,259200.0,False,2380.502,0.3125,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3398255770922054819,2019-04-18 00:49:51.569,259200.0,False,2991.569,0.3125,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
new_dtypes = {col: ('datetime64' if col=='created' else ('object' if col=='ref_hash' else ('float64' if col in ['in_seconds', 'last_seen', 'hour_day_dist'] else ('bool' if col in ['status_censored'] else 'int16')))) for col in convertions.columns.tolist()}
convertions = convertions.astype(new_dtypes, copy=False)
features = np.setdiff1d(convertions.columns, ['in_seconds', 'status_censored'] ).tolist()
corr = convertions[features].corr()
#remove uncorrelated columns (less than: -0.95)
corr_columns = np.setdiff1d(convertions.columns, corr.loc[(corr<-0.95).any()].index.tolist() ).tolist()
if not('created' in corr_columns):
    corr_columns.append('created')
if not('ref_hash' in corr_columns):
    corr_columns.append('ref_hash')
if not('in_seconds' in corr_columns):
    corr_columns.append('in_seconds')
if not('last_seen' in corr_columns):
    corr_columns.append('last_seen')
if not('status_censored' in corr_columns):
    corr_columns.append('status_censored')
if not('hour_day_dist' in corr_columns):
    corr_columns.append('hour_day_dist')

convertions = convertions[corr_columns]
new_dtypes = {col: ('datetime64' if col=='created' else ('object' if col=='ref_hash' else ('float64' if col in ['in_seconds', 'last_seen', 'hour_day_dist'] else ('bool' if col in ['status_censored'] else 'int16')))) for col in convertions.columns.tolist()}
convertions.to_csv('data/convertions_merged_50.csv', index=False) 
new_dtypes


{'application_id=116': 'int16',
 'application_id=117': 'int16',
 'application_id=121': 'int16',
 'application_id=122': 'int16',
 'application_id=126': 'int16',
 'application_id=128': 'int16',
 'application_id=133': 'int16',
 'application_id=135': 'int16',
 'application_id=14': 'int16',
 'application_id=140': 'int16',
 'application_id=145': 'int16',
 'application_id=147': 'int16',
 'application_id=155': 'int16',
 'application_id=158': 'int16',
 'application_id=161': 'int16',
 'application_id=167': 'int16',
 'application_id=185': 'int16',
 'application_id=187': 'int16',
 'application_id=188': 'int16',
 'application_id=19': 'int16',
 'application_id=190': 'int16',
 'application_id=197': 'int16',
 'application_id=20': 'int16',
 'application_id=21': 'int16',
 'application_id=210': 'int16',
 'application_id=226': 'int16',
 'application_id=230': 'int16',
 'application_id=232': 'int16',
 'application_id=256': 'int16',
 'application_id=26': 'int16',
 'application_id=263': 'int16',
 'application